In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/train-balanced-sarcasm.csv'
data_train = pd.read_csv(path_to_file)

Mounted at /content/drive


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/test-balanced-final.csv'
data_train = pd.read_csv(path_to_file)

Mounted at /content/drive


DARYL

In [ ]:
!pip install vaderSentiment
!pip install emoji
import pandas as pd
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
import emoji

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

### removing duplicate rows

data_train.drop_duplicates(keep='first',inplace=True)
data_train.drop_duplicates(keep='first',inplace=True)
print('Number of Duplicate rows in the dataset:',len(data_train[data_train.duplicated() == True]))


## score calculated as number of upvotes - downvotes
data_train['score'] = data_train['ups'] - data_train['downs']

def compute_basic_features(text1,text2):
    ###computes basic features between child and parent comment
       ### 1. child_len: length of the child comment
       ### 2. parent_len: length of the parent comment
       ### 3. distance_cp: distance between both the child and parent statement.More the distance more the constrast in
       ### sentiments. 
    text1=text1.tolist()
    text2=text2.tolist()
    sent_id = SentimentIntensityAnalyzer()
    child_len=[]
    parent_len=[]
    distance_cp=[]
    for com1,com2 in zip(text1,text2):
        token1 = com1.split()
        token2 = com2.split()
        # length of the  child
        child_length = len(token1)
        child_len.append(child_length)
        # length of the  parent
        parent_length = len(token2)
        parent_len.append(parent_length)
        score1 =  sent_id.polarity_scores(com1)['compound']
        score2 =  sent_id.polarity_scores(com2)['compound']
        distance = abs(score1-score2)
        distance_cp.append(distance)
    return child_len,parent_len,distance_cp
        
    
data_train['comment']=data_train['comment'].fillna("")
data_train['parent_comment']=data_train['parent_comment'].fillna("")

new_basic=compute_basic_features(data_train['comment'],data_train['parent_comment'])
#creating child_len, parent_len and distance_cp features
data_train['child_len']=new_basic[0]
data_train['parent_len']=new_basic[1]
data_train['distance_cp']=new_basic[2]


sent_id=SentimentIntensityAnalyzer()
exclude = set(string.punctuation)
def compute_advanced_features(text):
    '''Function will compute all the 11 features for every sentence passed as input'''
    tokens = text.split()
    ## noun_verb_c
    noun=['NN', 'NNS', 'NNP', 'NNPS']
    verb=['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]
    intensifier_list = ['absolutely','amazingly','astoundingly','at all','awful','bitterly','bloody','completely','crazy','dead','dreadfully',\
                    'colossally','especially','exceptionally','excessively','extremely','extraordinarily','fantastically','frightfully','fully',\
                    'hella','incredibly','insanely','literally','mad','mightily','outrageously',\
                    'particularly','phenomenally','precious','quite','radically','rather','real','really','remarkably','ridiculously',\
                    'so','somewhat','strikingly','super','supremely','terribly','terrifically','too','totally','unbelievably','veritable','very']
    emoji_list=["\U0001f600","\U0001f603","\U0001f604","\U0001f601","\U0001f606","\U0001f605","\U0001f923","\U0001f602","\U0001f642","\U0001f643"
                "\U0001f609","\U0001f60A","\U0001f607","\U0001f61b","\U0001f61c","\U0001f92a","\U0001f61d","\U0001f911"]
    interjection_list= ['hey','hurray','wow','brr','shh','oops','bravo','what','argh','oh dear','ouch','umm','phew','yoo-hoo','oh my','oh no',\
                        'eww','boo','snap','jeez','hey','yipee','orgh','bah','duh','congrats','yay','whoa','alas','hey','oh','eh','ah','zoinks',\
                        'hush','ahem','well','ahem','yum','bingo']

    ## initialize cs
    noun_count,verb_count = 0,0
    neg_int_c,pos_int_c = 0,0
    emoji_c,punctuation_c=0,0
    interjection_c,uppercase_c,repeated_letters_c=0,0,0
    flip_c,pos_c,neg_c=0,0,0
    previous_word_score={'pos':1,'neg':0}
    for word in tokens:
        if word not in stopwords:
            ### feature 1 :noun_verb_c
            tag = nltk.pos_tag(word_tokenize(word))[0][1]
            if tag in noun:
                noun_count+=1
            elif tag in verb:
                verb_count+=1
                
            ### feature2,3 neg_int_c,pos_int_c
            
            score = sent_id.polarity_scores(word)
            if word in intensifier_list:
                if score['neg']:
                    neg_int_c+=1
                elif score['pos']:
                    pos_int_c+=1
            ### feature 4 : emoji c 
            if word in emoji_list:
                emoji_c+=1   
            ### feature5 interjection_c
            if word in interjection_list:
                interjection_c+=1 
                
            ### feature6,7  : uppercase,repeated_letters
            if word == word.upper():
                uppercase_c+=1
                
            find_ = [(x.start(),x.end()) for x in re.finditer(r'(\w)\1\1', word)]
            
            if find_:
                repeated_letters_c+=1
            ## feature 8 : punctuation c 
            punctuation = set(string.punctuation)
            for i in  list(word):
                #print(i)
                if i in punctuation:
                    punctuation_c+=1
                    
            ## feature 9,10,11 - flip c ,pos c and neg c
            
            word_exclude_punc = ''.join(ch for ch in word if ch not in exclude)  
            score_new = sent_id.polarity_scores(word_exclude_punc)

            if score_new['pos']:
                pos_c+=1
                if previous_word_score['neg']:
                    flip_c+=1

            if score_new['neg']:
                neg_c+=1
                if previous_word_score['pos']:
                    flip_c+=1
            previous_word_score = score_new
            
    sum_nv = noun_count + verb_count
    return sum_nv,neg_int_c,pos_int_c,emoji_c,interjection_c,uppercase_c,repeated_letters_c,punctuation_c,flip_c,pos_c,neg_c

def mutating_df(dataframe):
    commentlist=dataframe['comment'].tolist()
    sum_nvl=[]
    neg_int_cl=[]
    pos_int_cl=[]
    emoji_cl=[]
    interjection_cl=[]
    uppercase_cl=[]
    repeated_letters_cl=[]
    punctuation_cl=[]
    flip_cl=[]
    pos_cl=[]
    neg_cl=[]
    for comment in commentlist:
        result=compute_advanced_features(comment)
        sum_nvl.append(result[0])
        neg_int_cl.append(result[1])
        pos_int_cl.append(result[2])
        emoji_cl.append(result[3])
        interjection_cl.append(result[4])
        uppercase_cl.append(result[5])
        repeated_letters_cl.append(result[6])
        punctuation_cl.append(result[7])
        flip_cl.append(result[8])
        pos_cl.append(result[9])
        neg_cl.append(result[10])
    data_train['sum_nv']=sum_nvl
    data_train['neg_int_c']=neg_int_cl
    data_train['pos_int_c']=pos_int_cl
    data_train['emoji_c']=emoji_cl
    data_train['interjection_c']=interjection_cl
    data_train['uppercase_c']=uppercase_cl
    data_train['repeated_letters_c']=repeated_letters_cl
    data_train['punctuation_c']=punctuation_cl
    data_train['flip_c']=flip_cl
    data_train['pos_c']=pos_cl
    data_train['neg_c']=neg_cl
    return dataframe

mutating_df(data_train)

from google.colab import files
data_train.to_csv('filename.csv') 
files.download('filename.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 9.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 8.6 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=dd4a2f65dbf591d1195946998c8222415cc8e48e180c26e34b7749cd04b27dd6
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Number of Duplicate rows in the dataset: 0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/filename.csv'
data_train = pd.read_csv(path_to_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


LE YAO

In [ ]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
import re
import spacy
from nltk.corpus import sentiwordnet as swn
from IPython.display import clear_output
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly
import nltk
import ssl
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import string
from nltk.tokenize import TweetTokenizer
string.punctuation
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
plotly.offline.init_notebook_mode (connected = True)

Edited_Comments = data_train['comment'].copy()
data_train['comment_without_stopwords'] = Edited_Comments
data_train['comment_without_stopwords'].replace('', np.nan, inplace=True)
data_train.dropna(subset=['comment_without_stopwords'], inplace=True)
data_train.reset_index(drop=True, inplace=True)

def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    # Removing double spaces if created
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))


def remove_stopwords_tokenize(data,name):
      
    def myTokenizer(txt):
        pattern = '(?<=\w)[^\w\s]*(?!\w)'
        txt = re.sub(pattern, '', txt)
        txt = TweetTokenizer().tokenize(txt)
        return txt

    def getting(sen):
        example_sent = sen
        
        filtered_sentence = [] 

        stop_words = set(stopwords.words('english')) 

        word_tokens = myTokenizer(example_sent)
        
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 
        
        return filtered_sentence
    # Using "getting(sen)" function to append edited sentence to data
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x

lemmatizer = WordNetLemmatizer()
def Lemmatization(data,name):
    def getting2(sen):
        
        example = sen
        output_sentence =[]
        word_tokens2 = word_tokenize(example)
        lemmatized_output = [lemmatizer.lemmatize(w) for w in word_tokens2]
        
        # Remove characters which have length less than 2  
        without_single_chr = [word for word in lemmatized_output if len(word) > 2]
        # Remove numbers
        cleaned_data_title = [word for word in without_single_chr if not word.isnumeric()]
        
        return cleaned_data_title
    # Using "getting2(sen)" function to append edited sentence to data
    x=[]
    for i in data[name].values:
        x.append(getting2(i))
    data[name]=x

# Using tokenizer and removing the stopwords
remove_stopwords_tokenize(data_train,'comment_without_stopwords')
# Converting all the texts back to sentences
make_sentences(data_train,'comment_without_stopwords')

#Edits After Lemmatization
final_Edit = data_train['comment_without_stopwords'].copy()
data_train["After_lemmatization"] = final_Edit

# Using the Lemmatization function to lemmatize the hotel data
Lemmatization(data_train,'After_lemmatization')
# Converting all the texts back to sentences
make_sentences(data_train,'After_lemmatization')

data_train.head(6)

pos=neg=obj=count=0

postagging = []

for review in data_train['After_lemmatization']:
    list = word_tokenize(review)
    postagging.append(nltk.pos_tag(list))

data_train['pos_tags'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0
    
    ###################################################################################
senti_score = []

for pos_val in data_train['pos_tags']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
data_train['senti_score'] = senti_score
print(data_train['senti_score'])

print(data_train.head)

overall=[]
for i in range(len(data_train)):
    if data_train['senti_score'][i]>= 0.05:
        overall.append('Positive')
    elif data_train['senti_score'][i]<= -0.05:
        overall.append('Negative')
    else:
        overall.append('Neutral')
data_train['Overall Sentiment']=overall
data_train.head(10)

data_train['comment_new'] = data_train['After_lemmatization'].copy()

from google.colab import files
data_train.to_csv('trainly1.csv') 
files.download('trainly1.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


0         0.375
1        -0.125
2         0.375
3         0.000
4        -0.250
          ...  
251585   -0.500
251586    0.000
251587    0.750
251588    0.000
251589    0.375
Name: senti_score, Length: 251590, dtype: float64
<bound method NDFrame.head of         Unnamed: 0  label                                            comment  \
0                0      0  Actually most of her supporters and sane peopl...   
1                1      0  They can't survive without an echo chamber whi...   
2                2      0             you're pretty cute yourself 1729 total   
3                3      0        If you kill me you'll crash the meme market   
4                4      0  I bet he wrote that last message as he was sob...   
...            ...    ...                                                ...   
251585      251603      1               Respect your elders you little snot.   
251586      251604      1  I'm just glad they won't be using taxpayer mon...   
251587      251605      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/trainly1.csv'
data_train = pd.read_csv(path_to_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# The following code creates a word-document matrix.
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(data_train['comment_new'].values.astype('U'))
df = pd.DataFrame(X.toarray(), columns = vec.get_feature_names())
df.head(3)

vect = CountVectorizer()
vect.fit(data_train['reviews_text_new'])
vect.get_feature_names()
# transform training data into a 'document-term matrix'
simple_train_dtm = vect.transform(data_train['reviews_text_new'])
print(simple_train_dtm)

### Creating a python object of the class CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
bow_counts = CountVectorizer(tokenizer= word_tokenize, # type of tokenization
                             ngram_range=(1,3)) # number of n-grams

bow_data = bow_counts.fit_transform(data_train['reviews_text_new'])

from google.colab import files
data_train.to_csv('trainly2.csv') 
files.download('trainly2.csv')

SI JING

In [ ]:
tfidf_comments = data_train['comment'].copy()
data_train['comments tfidf'] = tfidf_comments
data_train["comments tfidf"] = data_train["comments tfidf"].astype(str)
def create_tfidf_ngram_vectorizer(text_train, ngram_range = (1,1), **kwargs):
    vectorizer = TfidfVectorizer(ngram_range = ngram_range, **kwargs)
    vectorizer.fit(text_train)
    return vectorizer

for i in range(1,3):
    tfidf_igram_vectorizer = create_tfidf_ngram_vectorizer(data_train['comments tfidf'], ngram_range = (1,i))
    X_train_transformed = tfidf_igram_vectorizer.transform(data_train['comments tfidf'])
    
    base_classifier = sklearnClassifier(SGDClassifier(), X_train_transformed, y_train)

from google.colab import files
data_train.to_csv('trainsijing.csv') 
files.download('trainsijing.csv')

NameError: ignored

EN EN

LOG REGRESSION AND RANDOM FOREST

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/trainly1.csv'
data_train = pd.read_csv(path_to_file)

Mounted at /content/drive


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/filename1.csv'
data_test = pd.read_csv(path_to_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/trainbalancedfinal6k.csv'
data_train = pd.read_csv(path_to_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/My Drive/Colab Notebooks/testbalancedfinal2k.csv'
data_test = pd.read_csv(path_to_file)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import numpy as np

#split train and target class
y_train = data_train['label']
x_train = data_train.drop(['comment', 'author','subreddit', 'date', 'created_utc', 'parent_comment', 'comment_without_stopwords', 'After_lemmatization', 
                           'pos_tags', 'comment_new', 'Overall Sentiment', 'label', 'Index'], axis=1)
#logregression
param = {'C': [10**-2,10**-1,10**0,10**1,10**2]}
#test unbalanced
y_test = data_test['label']
x_test = data_test.drop(['comment', 'author','subreddit', 'date', 'parent_comment', 'comment_without_stopwords', 'After_lemmatization', 
                           'pos_tags', 'comment_new', 'Overall Sentiment', 'code', 'label', 'Index'], axis=1)




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
lrm=LogisticRegression()
rfm = RandomForestClassifier(n_estimators = 140, random_state = 42)
#cvv for logistic regression
scores = cross_val_score(lrm, x_trainstd, y_train, cv=5)
print(scores)
#cvv for random forest classifier
scores= cross_val_score(rfm,x_trainstd,y_train,cv=5)
print(scores)

[0.514735   0.5557     0.55112    0.54989    0.50485252]
[0.501435   0.354745   0.345695   0.39764    0.49859749]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
lrm=LogisticRegression()
rfm = RandomForestClassifier(n_estimators = 140, random_state = 42)
#cvv for logistic regression
scores = cross_val_score(lrm, x_trainstd, y_train, cv=5, scoring='f1_macro')
print(scores)
#cvv for random forest classifier
scores= cross_val_score(rfm,x_trainstd,y_train,cv=5, scoring='f1_macro')
print(scores)


[0.37003529 0.54392352 0.54979942 0.53180456 0.35767138]
[0.3339705  0.35458173 0.34569443 0.39744952 0.33276254]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
lrm=LogisticRegression()
rfm = RandomForestClassifier(n_estimators = 140, random_state = 42)
#cvv for logistic regression
scores = cross_val_score(lrm, x_trainstd, y_train, cv=5, scoring='f1_weighted')
print(scores)
#cvv for random forest classifier
scores= cross_val_score(rfm,x_trainstd,y_train,cv=5, scoring='f1_weighted')
print(scores)

[0.37090181 0.54413386 0.54972945 0.53154139 0.35679047]
[0.334929   0.35461119 0.34569267 0.39741888 0.33180951]


In [ ]:
##before redundancy
lr_model_all = LogisticRegression() # Logistic regression
lr_model_all.fit(x_train, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all.predict(x_test) # Class prediction

#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, test_pred_lr_all).ravel()
print((tn, fp, fn, tp))

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))

from sklearn.metrics import log_loss
print(log_loss(y_test, test_pred_lr_all))

(1, 125798, 0, 125791)
              precision    recall  f1-score   support

           0       1.00      0.00      0.00    125799
           1       0.50      1.00      0.67    125791

    accuracy                           0.50    251590
   macro avg       0.75      0.50      0.33    251590
weighted avg       0.75      0.50      0.33    251590

17.270199851681294


In [ ]:
#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

# Train the model on training data
rf.fit(x_train, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_test)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
x_teststd = scaler.transform(x_test)
##before redundancy
lr_model_all = LogisticRegression() # Logistic regression
lr_model_all.fit(x_trainstd, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all.predict(x_teststd) # Class prediction

#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, test_pred_lr_all).ravel()
print((tn, fp, fn, tp))

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))

from sklearn.metrics import log_loss
print(log_loss(y_test, test_pred_lr_all))

(103426, 22373, 92591, 33200)
              precision    recall  f1-score   support

           0       0.53      0.82      0.64    125799
           1       0.60      0.26      0.37    125791

    accuracy                           0.54    251590
   macro avg       0.56      0.54      0.50    251590
weighted avg       0.56      0.54      0.50    251590

15.782558046257074


In [ ]:
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
x_teststd = scaler.transform(x_test)

#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

# Train the model on training data
rf.fit(x_trainstd, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_teststd)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
x_trainnorm = scaler.fit_transform(x_train)
x_testnorm = scaler.transform(x_test)
##before redundancy
lr_model_all = LogisticRegression() # Logistic regression
lr_model_all.fit(x_trainnorm, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all.predict(x_testnorm) # Class prediction

#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, test_pred_lr_all).ravel()
print((tn, fp, fn, tp))

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))

(125798, 1, 125791, 0)
              precision    recall  f1-score   support

           0       0.50      1.00      0.67    125799
           1       0.00      0.00      0.00    125791

    accuracy                           0.50    251590
   macro avg       0.25      0.50      0.33    251590
weighted avg       0.25      0.50      0.33    251590



In [ ]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
x_trainnorm = scaler.fit_transform(x_train)
x_testnorm = scaler.transform(x_test)

#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

# Train the model on training data
rf.fit(x_trainnorm, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_testnorm)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_trainrs = scaler.fit_transform(x_train)
x_testrs = scaler.transform(x_test)
##before redundancy
lr_model_all = LogisticRegression(max_iter = 500) # Logistic regression
lr_model_all.fit(x_trainrs, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all.predict(x_testrs) # Class prediction

#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, test_pred_lr_all).ravel()
print((tn, fp, fn, tp))

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))

(103374, 22425, 92532, 33259)
              precision    recall  f1-score   support

           0       0.53      0.82      0.64    125799
           1       0.60      0.26      0.37    125791

    accuracy                           0.54    251590
   macro avg       0.56      0.54      0.50    251590
weighted avg       0.56      0.54      0.50    251590



In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_trainrs = scaler.fit_transform(x_train)
x_testrs = scaler.transform(x_test)

#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

# Train the model on training data
rf.fit(x_trainrs, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_testrs)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_trainmm = scaler.fit_transform(x_train)
x_testmm = scaler.transform(x_test)
##before redundancy
lr_model_all = LogisticRegression(max_iter=500) # Logistic regression
lr_model_all.fit(x_trainmm, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all.predict(x_testmm) # Class prediction

#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, test_pred_lr_all).ravel()
print((tn, fp, fn, tp))

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))

(110090, 15709, 102609, 23182)
              precision    recall  f1-score   support

           0       0.52      0.88      0.65    125799
           1       0.60      0.18      0.28    125791

    accuracy                           0.53    251590
   macro avg       0.56      0.53      0.47    251590
weighted avg       0.56      0.53      0.47    251590



In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_trainmm = scaler.fit_transform(x_train)
x_testmm = scaler.transform(x_test)

#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

# Train the model on training data
rf.fit(x_trainmm, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_testmm)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import feature_selection
scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
x_teststd = scaler.transform(x_test)

lr_model = LogisticRegression(penalty='l1', solver='liblinear')

gs_model = GridSearchCV(estimator=lr_model, param_grid=param, pre_dispatch=None)
gs_model.fit(x_trainstd, y_train)

# Train a LR model with best parameters
model = LogisticRegression(**gs_model.best_params_, penalty='l1', solver='liblinear')
model.fit(x_trainstd, y_train)

coef = model.coef_[0]
imp_features = pd.Series(x_train.columns)[list(coef!=0)]
X_train = x_train[imp_features]
X_test = x_test[imp_features]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##after redundancy
lr_model_all1 = LogisticRegression() # Logistic regression
lr_model_all1.fit(X_train, y_train) # Fitting a logistic regression model

## Predicting the output
test_pred_lr_all = lr_model_all1.predict(X_test) # Class prediction

from sklearn.metrics import classification_report
# Print a classification report
print(classification_report(y_test,test_pred_lr_all))


In [ ]:
#random forest
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 140, random_state = 42, verbose = 2)

scaler = StandardScaler()
x_trainstd = scaler.fit_transform(x_train)
x_teststd = scaler.transform(x_test)
# Train the model on training data
rf.fit(x_trainstd, y_train)

# Use the forest's predict method on the test data
predictions = rf.predict(x_teststd)

#confusion matrix
tn, fp, fn, tp = confusion_matrix(predictions, y_test).ravel()
print((tn, fp, fn, tp))
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))